In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/data/data_clef.zip /content
!unzip data_clef.zip -d /content

把每个knowledge embedding cleftrain，cleftest，clefVal分别移动到training，testing，validation，然后把名字都改成knowledge_embeddings.

In [3]:
!cp -r /content/drive/MyDrive/data/knowledge_embeddings_cleftrain.zip /content
!unzip knowledge_embeddings_cleftrain.zip -d /content/data_clef/training

^C
Archive:  knowledge_embeddings_cleftrain.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of knowledge_embeddings_cleftrain.zip or
        knowledge_embeddings_cleftrain.zip.zip, and cannot find knowledge_embeddings_cleftrain.zip.ZIP, period.


In [4]:
!cp -r /content/drive/MyDrive/data/knowledge_embeddings_cleftest.zip /content
!unzip knowledge_embeddings_cleftest.zip -d /content/data_clef/testing

Archive:  knowledge_embeddings_cleftest.zip
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/122.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/89.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/331.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/276.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/289.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/236.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/227.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/469.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/175.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/70.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/191.npy  
  inflating: /content/data_clef/testing/knowledge_embeddings_cleftest/329.npy 

In [5]:
!cp -r /content/drive/MyDrive/data/knowledge_embeddings_clefVal.zip /content
!unzip knowledge_embeddings_clefVal.zip -d /content/data_clef/validation

^C
Archive:  knowledge_embeddings_clefVal.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of knowledge_embeddings_clefVal.zip or
        knowledge_embeddings_clefVal.zip.zip, and cannot find knowledge_embeddings_clefVal.zip.ZIP, period.


In [6]:
!mv /content/data_clef/training/knowledge_embeddings_cleftrain /content/data_clef/training/knowledge_embeddings
!mv /content/data_clef/testing/knowledge_embeddings_cleftest /content/data_clef/testing/knowledge_embeddings
!mv /content/data_clef/validation/knowledge_embeddings_clefVal /content/data_clef/validation/knowledge_embeddings

mv: cannot stat '/content/data_clef/training/knowledge_embeddings_cleftrain': No such file or directory
mv: cannot stat '/content/data_clef/validation/knowledge_embeddings_clefVal': No such file or directory


In [ ]:
%cd /content/drive/MyDrive/VQACode

/content/drive/MyDrive/VQACode


记得把check_point_clef移到文件夹

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data_overall import DataLoader, ClefDataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks




In [ ]:
num_layers=2
d_model=512
num_heads=8
dff=2048
maximum_position_encoding=10000
EPOCHS = 50
batch_size = 64
cnn_type = 'resnet'
embedding = 'bioelmo'  # choose from ['w2v', 'bioelmo', 'biobert', 'bluebert', 'large_biobert', 'elmo', 'bert']
data_augmentation = True

In [ ]:
####### DO NOT CHANGE VALUES OF THIS BLOCK IF YOU ARE NOT THE DEVELOPER ##########

check_point_path = './check_point_clef/transformer/overall/' + embedding +'/' + cnn_type + '_' + str(num_layers)
saving_folder = './overall_results_clef/transformer/' + embedding + '/'
save_result_path = saving_folder + cnn_type + '_' + str(num_layers) + '.csv'

kn_input = 155
emb_size = 1024
pe_output = 36 + 1 
MAX_LENGTH = pe_output
if cnn_type == 'inception':
    img_shape = [299, 299]
    img_padding = tf.TensorShape([299, 299, 3])
if cnn_type in ['resnet', 'resnet_v2', 'dense_net', 'vgg19']:
    img_shape = None
    img_padding = tf.TensorShape([224, 224, 3])

if embedding == 'bioelmo':
    pe_input = 11
# elif embedding == 'elmo':
#     pe_input = 42
# elif embedding == 'biobert':
#     pe_input = 72
#     emb_size = 768
# elif embedding == 'bluebert':
#     pe_input = 69
# elif embedding == 'large_biobert':
#     pe_input = 60  
# elif embedding == 'w2v':
#     pe_input = 48
#     emb_size = 200
# elif embedding == 'bert':
#     pe_input = 72
#     emb_size = 1024
# else:
#     raise TypeError("Wrong embedding type")
    
if data_augmentation:
    aug = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.RandomFlip(),
                               tf.keras.layers.experimental.preprocessing.RandomRotation(0.05)])

In [ ]:
train_loader = DataLoader('/content/data_clef/training', embedding)
train_dataset, train_tokenizer = train_loader.create_dataset('open_ended')
batched_train_set = train_dataset.padded_batch(batch_size, padded_shapes=((img_padding, tf.TensorShape([pe_input, emb_size]), tf.TensorShape([kn_input, 1024])),
                                                                  tf.TensorShape([pe_output-1]), []), drop_remainder=True)

QA: 3200
yes_no: 12792
open_ended 12792
Load: open_ended


In [ ]:
test_loader = DataLoader('/content/data_clef/testing', embedding)
test_dataset, test_tokenizer = test_loader.create_dataset('open_ended')
batched_test_set = test_dataset.padded_batch(1, padded_shapes=((img_padding, tf.TensorShape([pe_input, emb_size]), tf.TensorShape([kn_input, 1024])),
                                                         tf.TensorShape([pe_output-1]), []), drop_remainder=True)

QA: 125
yes_no: 500
open_ended 500
Load: open_ended


In [ ]:
val_loader = DataLoader('/content/data_clef/validation', embedding)
val_dataset, val_tokenizer = val_loader.create_dataset('open_ended')
batch_val_set = val_dataset.padded_batch(1, padded_shapes=((img_padding, tf.TensorShape([pe_input, emb_size]), tf.TensorShape([kn_input, 1024])),
                                                                    tf.TensorShape([pe_output-1]), []), drop_remainder=True)

QA: 500
yes_no: 202
open_ended 2000
Load: open_ended


In [ ]:
vocab_size=len(train_tokenizer.index_word) + 1
print(vocab_size)

2004


### 

In [ ]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
@tf.function()
def train_step(img, question, kn, tar):
    if data_augmentation:
        img = aug(img)
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(question, tar_inp)
    with tf.GradientTape() as tape:
        predictions, _ = transformer(question, img, kn, tar_inp,
                                     True,
                                     enc_padding_mask,
                                     combined_mask,
                                     dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(tar_real, predictions)

In [ ]:
def evaluate(question, img, kn):
    end_token = tf.constant(train_tokenizer.texts_to_sequences(['<end>']), tf.int32)
    output = dec_input = tf.expand_dims([train_tokenizer.word_index['<start>']], 0)
    for i in range(MAX_LENGTH):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            question, output)
        predictions, attention_weights = transformer(question,
                                                     img,
                                                     kn,
                                                     output,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)

        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        if predicted_id == end_token:
            return tf.squeeze(output, axis=0), attention_weights
        output = tf.concat([output, predicted_id], axis=-1)
    return tf.squeeze(output, axis=0), attention_weights

In [ ]:
################ADD#################################################################################
def get_score(batch_data_set, csv_saving_path): 
    true_answers_list = []
    predicted_answers_list = []
    ques_id_list = []
    for (batch, (img_question, target, ques_id)) in enumerate(batch_data_set):
        target = target.numpy()
        target = target[0]
        true_answer = []
        for i in target:
            if i == 0:
                break
            else:
                true_answer.append(tokenizer.index_word[i])
        true_answer = " ".join(true_answer[1: -1])
        prediction, attention = evaluate(img_question[1], img_question[0], img_question[2])
        p = prediction.numpy()
        # print('an1:',p)
        predict_answer = [tokenizer.index_word[i] for i in p][1:]
        # print('an2:',predict_answer)
        predict_answer = " ".join(predict_answer)
        true_answers_list.append(true_answer)
        predicted_answers_list.append(predict_answer)
        ques_id_list.append(ques_id)
    # print('answer list:',predicted_answers_list)
    data = {"true answer": true_answers_list, "predicted answer": predicted_answers_list, "ques_id": ques_id_list}
    df = pd.DataFrame(data)
    if not pathlib.Path(saving_folder).exists():
        pathlib.Path(saving_folder).mkdir(parents=True, exist_ok=True)
    df.to_csv(csv_saving_path)
    # print("complete writing", csv_saving_path)
    return AnswerEvaluator(csv_saving_path).evaluate()
##################ADD#################################################################################

### 

In [ ]:
## restore check point 
# ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()
    train_loss.reset_states()
    train_accuracy.reset_states()
    for (batch, (img_question, tar, _)) in enumerate(batched_train_set):
        train_step(img_question[0], img_question[1], img_question[2], tar)
        # if batch % 50 == 0:
        #     print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
        #         epoch + 1, batch, train_loss.result(), train_accuracy.result()))
    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                        train_loss.result(),
                                        train_accuracy.result()))
 ##################Change#################################################################################
    # csv_saving_path = saving_folder + 'val' + str(epoch) + '.csv'
    # score = get_score(batch_val_set, csv_saving_path)
    # model_accuracy = score['Accuracy']
    # # if model_accuracy > accuracy:
    # print('Validation Accuracy',model_accuracy)
    # # ckpt_save_path = ckpt_manager.save()
    # # accuracy = model_accuracy            
##################Change#################################################################################

    # if (epoch + 1) % 2 == 0:
    #     ckpt_save_path = ckpt_manager.save()
    #     print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
    #                                                         ckpt_save_path))

    # print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
    #                                             train_loss.result(),
    #                                             train_accuracy.result()))

    # print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Loss 0.4563 Accuracy 0.0325
Epoch 2 Loss 0.2392 Accuracy 0.0564
Epoch 3 Loss 0.1890 Accuracy 0.0616
Epoch 4 Loss 0.1569 Accuracy 0.0657
Epoch 5 Loss 0.1309 Accuracy 0.0694
Epoch 6 Loss 0.1114 Accuracy 0.0722
Epoch 7 Loss 0.0969 Accuracy 0.0744
Epoch 8 Loss 0.0849 Accuracy 0.0763
Epoch 9 Loss 0.0756 Accuracy 0.0776
Epoch 10 Loss 0.0682 Accuracy 0.0787
Epoch 11 Loss 0.0630 Accuracy 0.0795
Epoch 12 Loss 0.0596 Accuracy 0.0798
Epoch 13 Loss 0.0548 Accuracy 0.0807
Epoch 14 Loss 0.0526 Accuracy 0.0808
Epoch 15 Loss 0.0506 Accuracy 0.0812
Epoch 16 Loss 0.0479 Accuracy 0.0816
Epoch 17 Loss 0.0470 Accuracy 0.0818
Epoch 18 Loss 0.0456 Accuracy 0.0820
Epoch 19 Loss 0.0452 Accuracy 0.0819


KeyboardInterrupt: ignored

In [ ]:
true_answers_list = []
predicted_answers_list = []
ques_id_list = []
print('Start predicting...')
for (batch, (img_question, target, ques_id)) in enumerate(batched_test_set):
    target = target.numpy()
    target = target[0]
    true_answer = []
    for i in target:
        if i == 0:
            break
        else:
            true_answer.append(test_tokenizer.index_word[i])
    true_answer = " ".join(true_answer[1: -1])
    
#     print(img_question[1].shape, img_question[0].shape)
    prediction, attention = evaluate(img_question[1], img_question[0], img_question[2])
    p = prediction.numpy()
    predict_answer = [train_tokenizer.index_word[i] for i in p][1:]
    predict_answer = " ".join(predict_answer)
    true_answers_list.append(true_answer)
    predicted_answers_list.append(predict_answer)
    ques_id_list.append(ques_id)
    print("predicted answer: " + str(batch), end='\r', flush=True)

Start predicting...


In [ ]:
data = {"true answer": true_answers_list, "predicted answer": predicted_answers_list, "ques_id":ques_id_list}
df = pd.DataFrame(data)
if not pathlib.Path(saving_folder).exists():
    pathlib.Path(saving_folder).mkdir(parents=True, exist_ok=True)
name = save_result_path
df.to_csv(name)
print("complete writing", name)

complete writing ./overall_results_clef/transformer/bioelmo/resnet_2.csv


In [ ]:
scores = AnswerEvaluator(name).evaluate()

{'testlen': 1114, 'reflen': 1181, 'guess': [1114, 614, 370, 186], 'correct': [612, 283, 162, 63]}
ratio: 0.9432684165953062
Accuracy: 57.31
Exact Match: 53.2
F1 Score: 56.43
BLEU-1: 0.52
BLEU-2: 0.47
BLEU-3: 0.45
BLEU-4: 0.41


In [ ]:
import json
with open('/content/data_clef/testing/jsons/open_ended.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
data[0]

{'Answers': 'cta - ct angiography',
 'Images': 'synpic54082',
 'Question_Id': 1,
 'Questions': 'what modality is shown?'}

In [ ]:
for d in data:
  if 'synpic39867' == d['Images']:
    print(d)

{'Question_Id': 172, 'Images': 'synpic39867', 'Questions': 'in which plane is the mri displayed?', 'Answers': 'axial'}


# Predict

In [ ]:
import numpy as np
image_size = [224, 224]
def load_and_preprocess_image(path):
    image = tf.io.read_file(path) # read image file
    image = tf.image.decode_jpeg(image, channels=3) # decode image
    image = tf.image.resize(image, image_size)
    return image
def load_question_features(path):
    return np.load(path)

In [ ]:
image_id = 'synpic39867'
ques_embd = 'bioelmo/'
question_id = '172'

In [ ]:
img_input = tf.convert_to_tensor(load_and_preprocess_image('/content/data_clef/testing/pic/'+str(image_id)+'.jpg'))
ques_input = tf.convert_to_tensor(load_question_features('/content/data_clef/testing/ques_embeddings/'+str(ques_embd)+str(question_id)+'.npy'))
kg_input = tf.convert_to_tensor(load_question_features('/content/data_clef/testing/knowledge_embeddings/'+str(question_id)+'.npy'))

ques_input = tf.pad(ques_input,[[0,pe_input-ques_input.shape[0]],[0,0]],"CONSTANT")
kg_input = tf.pad(kg_input,[[0,kn_input-kg_input.shape[0]],[0,0]],"CONSTANT")

img_input = tf.expand_dims(img_input, axis=0)
ques_input = tf.expand_dims(ques_input, axis=0)
kg_input = tf.expand_dims(kg_input, axis=0)

# Answer
prediction, attention = evaluate(ques_input, img_input, kg_input)

p = prediction.numpy()
predict_answer = [train_tokenizer.index_word[i] for i in p][1:]
predict_answer = " ".join(predict_answer)
print(predict_answer)

axial
